In [3]:
tcf(filename::String) = (@__DIR__) * "\\" * filename;

# Исследование напряженного состояния кронштейна

В рамках данного урока м рассмотрим трехмерную задачу о нагружении кронштейна.
В прошлых уроках мы строили геометрию вручную при помощи инструментов gmsh, однако для более сложной геометрии такой подход не слишком удобен, и гораздо проще воспользоваться какой-либо CAD системой, позволяющей строить 3D геометрию.
Расчетная схема представлена на рисунке 1.

В данном случае, геометрия была построена в [бесплатном пакете FreeCAD]/[отечественном пакете Компас3D] и экспортирована в формат `*.stp`. Полученая геометрия показана на рисунке ниже.

Для приложения нагрузки цилиндрическая поверхность была разделена на две половины и нижня окрашена в цвет (0, 100, 0). В дальнейшем мы буем использовать цвет поверхности для быстрого поиска данной поверхности при загрузке ее gmsh. Аналогичным образом поверхность, которая фиксируется, помечена цветом (100, 0, 0).

## Импорт геометрии подготовка КЭ сетки

Импорт готовой геометрии в gmsh осуществляется при помощи функции `gmsh.model.occ.importShapes(file_name)`
Далее необходимо выявить индексы поверхностей, которые были помечены при моделировании. Для этого можно поместить их в словарь, ключем которого будет являтся цвет, а значением - индекс поверхности. После этого данным поверхностям следует задать метки, для чего необходимо поместить их в физический группы при помощи команды `gmsh.model.addPhysicalGroup(dim, ents, id, label)`, где:
- `dim::Integer` - размерность физической группы (0 - точки, 1 - ребра, 2 - поверхности, 3 - объемы);
- `ents::Vector{<:Integer}` - вектор из индексов объектов, которые планиурется добавить к физической группе;
- `id::Integer` - индекс создаваемой группы (в случае, если будет передано `-1` - индекс проставится автоматически);
- `label::String` - метка создаваемой группы
Кроме того, необходимо создать физическую группу для основного тела, для того чтобы в дальнейшем gmsh включил его в задание на построение сетки. 


In [37]:
using Gmsh

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)

gmsh.model.add("model")


path = "lug.stp" |> tcf
v = gmsh.model.occ.importShapes(path)


gmsh.model.occ.synchronize()

faces = Dict()
for plane in gmsh.model.getEntities(2)
    faces[gmsh.model.getColor(plane...)] = plane[2];
    # gmsh.model.addPhysicalGroup(2, [plane[2]], -1, "$(plane[2])");
end


d3 = gmsh.model.getEntities(3)
gmsh.model.addPhysicalGroup(3, [d3[1][2]], -1, "domain")

d2 = gmsh.model.getEntities(2)
gmsh.model.addPhysicalGroup(2, [faces[(0,100,0,255)]], -1, "loaded")
gmsh.model.addPhysicalGroup(2, [faces[(100,0,0,255)]], -1, "fixed")

gmsh.model.occ.synchronize()

Далее необходимо задать параметры для построения сетки. В Gmsh предусмотрено несколько вариантов для задания параметров сетки. Один из них - при мпомощи команды gmsh.options.setNumber("Parameter")В качестве параметров можно задать:
 - минимальный и максимальный размеры элемента
 - количество элементов на криволинейных поверхностях
 - алгоритм триангуляции
 - и т.д.
Подробный список параметров можно посмотреть в руководстве gmsh.

После этого необходимо синхронизировать модель, выполнить генерацию сетки, сохранить результат в файл и завершить сессию gmsh.

In [38]:
gmsh.option.setNumber("Mesh.MeshSizeMax", 3.0)          #Максимальный размер элементов внутри всей модели - 3мм
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 50) #колличество элементов на 2pi радиан

gmsh.model.mesh.generate(3)
msh_file = "lug.msh" |> tcf;
gmsh.write(msh_file)

# gmsh.fltk.run()

gmsh.finalize();

## Создание модели в Gridap

В этот раз, в отличие от предыдущего урока, воспользуемся встроиным в Gridap функционалом для построения конечных элементов.

Мы уже упомянали, что напряжения и деформации являются тензорами []. Наиболее простой представить   

In [ ]:
const E = 2.1e5
const ν = 0.3
const λ = (E*ν)/((1+ν)*(1-2*ν))
const μ = E/(2*(1+ν))

σ(ε) = λ*tr(ε) + 2*μ*ε